Importing necessary Libraries for dataframes and for classifier

In [0]:
import numpy as np
import pandas as pd
from scipy.io import arff
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import urllib.request
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras import layers
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


Declaring the column header

In [0]:
col = np.array(['Attr1', 'Attr2', 'Attr3', 'Attr4', 'Attr5', 'Attr6', 'Attr7', 'Attr8',
       'Attr9', 'Attr10', 'Attr11', 'Attr12', 'Attr13', 'Attr14', 'Attr15',
       'Attr16', 'Attr17', 'Attr18', 'Attr19', 'Attr20', 'Attr21', 'Attr22',
       'Attr23', 'Attr24', 'Attr25', 'Attr26', 'Attr27', 'Attr28', 'Attr29',
       'Attr30', 'Attr31', 'Attr32', 'Attr33', 'Attr34', 'Attr35', 'Attr36',
       'Attr37', 'Attr38', 'Attr39', 'Attr40', 'Attr41', 'Attr42', 'Attr43',
       'Attr44', 'Attr45', 'Attr46', 'Attr47', 'Attr48', 'Attr49', 'Attr50',
       'Attr51', 'Attr52', 'Attr53', 'Attr54', 'Attr55', 'Attr56', 'Attr57',
       'Attr58', 'Attr59', 'Attr60', 'Attr61', 'Attr62', 'Attr63', 'Attr64',
       'class'])

1.This function fetches the data from given URL. Then convert the string into list and then to dataframes

2.This will return the dataframe

In [0]:
def data_fetching(url):
  data = urllib.request.urlopen(url).read().decode('utf-8')
  data=data.replace('?','0')
  data_split = data.split('\n')
  data_split1 = data_split[70:]

  df = pd.DataFrame()
  del data_split1[-1]
  del data_split1[-1]
  del data_split1[-1]
  for i in data_split1:
    j = i.split(',')
    j=[float(x) for x in j]    
    m = pd.DataFrame(j,).T
    df = df.append(m,ignore_index=True)
  return df

  



We are fetching dataframes for each year using the function described above by passing the URL of each year dataset

In [0]:
url1 = "https://raw.githubusercontent.com/pragnabodipudi1/NeuralNetworks/master/1year.arff"
df1 = data_fetching(url1)
url2 = "https://raw.githubusercontent.com/pragnabodipudi1/NeuralNetworks/master/2year.arff"
df2 = data_fetching(url2)
url3 = "https://raw.githubusercontent.com/pragnabodipudi1/NeuralNetworks/master/3year.arff"
df3 = data_fetching(url3)
url4 = "https://raw.githubusercontent.com/pragnabodipudi1/NeuralNetworks/master/4year.arff"
df4 = data_fetching(url4)
url5 = "https://raw.githubusercontent.com/pragnabodipudi1/NeuralNetworks/master/5year.arff"
df5 = data_fetching(url5)

We are appending the the different frames(fetched for each year) into single frame

We are also giving the name to columns

In [0]:
df_final = pd.DataFrame()
df_final=df_final.append(df1,ignore_index=True)
df_final=df_final.append(df2,ignore_index=True)
df_final=df_final.append(df2,ignore_index=True)
df_final=df_final.append(df4,ignore_index=True)
df_final=df_final.append(df5,ignore_index=True)


df_final.columns=col


There are so many missing values or zeros across the dataframe. Trying to count no of missing values in each column

In [0]:
df = df_final
df = df.drop(columns=['class'])
df = df.where(df>0)
df.isnull().sum().sort_values(ascending=False).head(5)

Attr6     27324
Attr5     22081
Attr59    20056
Attr37    18791
Attr48    18050
dtype: int64

Dropping the columns with most missed values from dataframe 

In [0]:
df_final=df_final.drop(columns=['Attr6','Attr5','Attr59','Attr37','Attr48'])

Replacing the available zeros with mean values 

In [0]:
df_final.fillna(df.mean(),inplace=True)
print(df_final.shape)
X = df.iloc[:, [0, 59]]
y = df.iloc[:, 59]
df = df_final
print(df.head(10))

(43060, 60)
      Attr1     Attr2     Attr3    Attr4  ...    Attr62   Attr63    Attr64  class
0  0.209120  0.499880  0.472250   1.9447  ...  107.3500   3.4000   60.9870    0.0
1  0.248660  0.695920  0.267130   1.5548  ...  134.2700   2.7185    5.2078    0.0
2  0.081483  0.307340  0.458790   2.4928  ...   86.4350   4.2228    5.5497    0.0
3  0.187320  0.613230  0.229600   1.4063  ...  127.2100   2.8692    7.8980    0.0
4  0.228220  0.497940  0.359690   1.7502  ...   88.4440   4.1269   12.2990    0.0
5  0.111090  0.647440  0.289710   1.4705  ...  129.5500   2.8173   18.3520    0.0
6  0.532320  0.027059  0.705540  53.9540  ...    7.4503  48.9910    2.3217    0.0
7  0.009020  0.632020  0.053735   1.1263  ...  116.5000   3.1330    2.5603    0.0
8  0.124080  0.838370  0.142040   1.1694  ...  144.6300   2.5236  107.6700    0.0
9  0.240010  0.443550  0.188350   1.4400  ...   32.9280  11.0850   12.3690    0.0

[10 rows x 60 columns]


 Here data is imbalanced.Classification using class-imbalanced data is biased in favor of the majority class. The bias is even larger for high-dimensional data, where the number of variables greatly exceeds the number of samples.There are so many techniques to overcome the problem. Popular among them is SMOTE(Synthetic Minority Oversampling Technique).  

In [0]:
X = df.drop('class', axis=1)
y = np.array(df['class'].tolist()) 
#smote
smote = SMOTE(ratio='auto' , random_state=42, k_neighbors=10)
dfi_features_res, dfi_label_res = smote.fit_sample(X, y)
print(dfi_features_res.shape)
print(dfi_label_res.shape)
count_0 =0
count_1=0
for c in dfi_label_res:
  if(c == 0):
    count_0= count_0+1
  else:
    count_1 = count_1+1
print("count_1",count_1)
print("count_0",count_0)


Here we are using different classifies to validate the data.
We are declaring different classifiers

1.Gaussian Naïve Bayes, 2.Adaptive Boosting, 3.Logistc Regression 4. Random forest classifier 5.neural network 6. k-nearest neighbors algorithm 7.Decision Tree Classifier 8. X gradient boosting   



In [0]:
#hussy
gnb_classifier = GaussianNB()
ada_boost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=200)
#Rakshith
lr_classifier = LogisticRegression(penalty = 'l1', random_state = 0)
rfc = RandomForestClassifier()
#Pragna
dt_classifier = DecisionTreeClassifier(random_state=42)
xgb_classifier = XGBClassifier()
#Sai Vinay
knn_classifier = KNeighborsClassifier(n_neighbors=1,n_jobs=1)
#Deep Neural Network implemented at the bottom

This function is used to implement k-Fold Cross-Validating Neural Networks

To train a classifier network

To display training and testing results

to display convolution matrix



In [0]:
def kfold(classifier12):
  scores=[]
  #dfi_features_res, dfi_label_res
  kf = KFold(n_splits=10, shuffle=False, random_state=42)
  X_train = []
  y_train = []
  X_test = []
  y_test = []
  for train_index, test_index in kf.split(dfi_features_res):
    X_train, X_test, y_train, y_test = dfi_features_res[train_index], dfi_features_res[test_index], dfi_label_res[train_index], dfi_label_res[test_index]  
    classifier12.fit(X_train, y_train)
    scores.append(classifier12.score(X_test, y_test))
    print(scores)
 
  print(np.mean(scores))
  
  y_pred=cross_val_predict(classifier12, dfi_features_res, dfi_label_res, cv=kf)
  accuracy = accuracy_score(y_pred.astype(int), dfi_label_res.astype(int))
  print(accuracy)


To train a classifier network without k-fold

To display training and testing results

to display convolution matrix

In [0]:
# With out kfold
def WithOut_KFold(classifier12):
  X_train_S, X_test_S, y_train_S, y_test_S = train_test_split(dfi_features_res, dfi_label_res,random_state=1)
  classifier12.fit(X_train_S,y_train_S)
  predictions = classifier12.predict(X_test_S)
  confusion_matrix(y_test_S, predictions)
  print(classification_report(y_test_S, predictions))
  confusion_matrix(y_test_S, predictions)

Showing results for X gradient boosting

In [0]:
kfold(xgb_classifier)
WithOut_KFold(xgb_classifier)

Showing results for Decision Tree Classifier 

In [0]:
kfold(dt_classifier)
WithOut_KFold(dt_classifier)

Showing results for Gaussian Naïve Bayes

In [0]:
kfold(gnb_classifier)
WithOut_KFold(gnb_classifier)

 Showing results for Adaptive Boosting

In [0]:
kfold(ada_boost)
WithOut_KFold(ada_boost)

Showing results for Logistic Regression

In [0]:
kfold(lr_classifier)
WithOut_KFold(lr_classifier)


Showing results for Random forest classifier

In [0]:
kfold(rf_classifier)
WithOut_KFold(rf_classifier)

Showing results for KNN classifier

In [0]:
X_train, X_test, y_train, y_test = train_test_split(dfi_features_res, dfi_label_res,random_state=1)

#Measuring the Accuracy of the model by taking different values of k.
k_range = (1,26)
scores={}
scores_list=[]

for k in k_range:
  knn = KNeighborsClassifier(n_neighbors= k ) 
  knn.fit(X_train, y_train)
  y_pred = knn.predict(X_test)
  scores[k] = metrics.accuracy_score(y_test, y_pred)
  scores_list.append(metrics.accuracy_score(y_test, y_pred))

#Plotting the Accuracy of the odel for each k which we have computed above.
plt.plot(k_range, scores_list)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

#We have finally fixed our k as 1 and fit our data into it.
classifier = KNeighborsClassifier(n_neighbors=1,n_jobs=1)  
classifier.fit(X_train, y_train)
target_predict = classifier.predict(X_test)

#Printing the Confusion matrix and Test Accuracy of the Model.
print(confusion_matrix(y_test, target_predict))
print("Test Accuracy: ", accuracy_score(y_test, target_predict))

Showing results for Deep Learning Neural network

In [0]:
#Coverting our class labels into one hot encoding
dfi_label_res = to_categorical(dfi_label_res)

#Splitting the dataset into training and testing.
X_train, X_test, y_train, y_test = train_test_split(dfi_features_res, dfi_label_res,random_state=1)

#Assigning our model as Sequential
model=Sequential()

#Adding a dense hidden layer to our network with input dim as number of columns
#Relu is used as an Activation function
model.add(Dense(X_train.shape[0],input_dim=X_train.shape[1],activation='relu'))
#Dropout technique is used to prevent overfitting of the data 
model.add(layers.Dropout(0.3))
#Added some more hidden layers
model.add(Dense(40,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(20,activation='relu'))
model.add(Dense(2,activation='softmax'))

#Compiled our model using loss and optimizer functions.
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#Finally our model is fit using our training dataset
model.fit(X_train,y_train,epochs=5,batch_size=800,verbose=1)


In [0]:
#Assigning our test results to a variable
predictions =  model.predict(X_test)

print("Predictions in one hot encoding :",predictions)
print("Displaying our test labels :",y_test)

#De-coding our one hot encoding into integers.
predictions = np.argmax(predictions, axis= 1)

#Displaying the Training and Testing Accuracy of our model
print("Train Accuracy: ", model.evaluate(X_train, y_train, verbose=2)[1])
print("Test Accuracy: ", model.evaluate(X_test,  y_test, verbose=2)[1])

#Converting our testing labels into integers from one hot encoding.
y_test= np.argmax(y_test, axis= 1)

print("Confusion Matrix :",(confusion_matrix(y_test, predictions)))